# Dusty box

This notebook contains analysis of the dusty box test for multigrain dust.

In [ ]:
import pathlib
import sys

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import plonk

In [ ]:
sys.path.insert(0, '../modules')
from multigrain import dustybox

## Time evolution

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox/time_evolution').expanduser()
_paths = sorted(list(root_directory.glob('*')))
paths = {p.name: p for p in _paths}
paths

### Calculate velocity differential time evolution

In [ ]:
data = dict()
exact1 = dict()
exact2 = dict()

for name, path in paths.items():
    try:
        print(f'Running analysis for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        data[name] = dustybox.calculate_differential_velocity(sim)
        exact1[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=True)
        exact2[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=False)
    except:
        print(f'Failed to load {name}')

### Plot time evolution

In [ ]:
fig = dustybox.plot_differential_velocity_all(data, exact1, exact2, figsize=(10, 6))

### Calculate velocity differential error

In [ ]:
error = dict()

for name, path in paths.items():
    print(f'Calculating error for {name}...')
    sim = plonk.load_sim(prefix='dustybox', directory=path)
    error[name] = dustybox.calculate_error(sim)

### Plot error

In [ ]:
fig = dustybox.plot_error_all(error, figsize=(10, 6))

## Stability

In [ ]:
# Parameter in Phantom patch: "comparing" dt_force and dt_drag
DTFORCE_TO_DTDRAG = 3.6

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox/stability').expanduser()
_paths = sorted(list(root_directory.glob('*')))
paths = {p.name: p for p in _paths}
paths

In [ ]:
dust_to_gas = sorted(list(set([float(key[4:8]) for key in paths.keys()])))
C_force = sorted(list(set([float(key[17:]) for key in paths.keys()])))
dtdrag_fac = [val * DTFORCE_TO_DTDRAG for val in C_force]

### Calculate velocity differential time evolution

In [ ]:
data = dict()
exact1 = dict()
exact2 = dict()

for name, path in paths.items():
    try:
        print(f'Running analysis for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        data[name] = dustybox.calculate_differential_velocity(sim)
        exact1[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=True)
        exact2[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=False)
    except:
        print(f'Failed to load {name}')

### Plot stability

In [ ]:
n_dust_to_gas = len(dust_to_gas)
fig = dustybox.plot_differential_velocity_all(data, exact1, exact2, ncols=4, figsize=(12, 10), transpose=True)
for ax, eps in zip(fig.axes[:n_dust_to_gas], dust_to_gas):
    ax.set_title(rf'$\epsilon = {eps:.2f}$')
for ax, fac in zip(fig.axes[::n_dust_to_gas], dtdrag_fac):
    ax.set_ylabel(rf'{fac:.4f}')

## Accuracy

In [ ]:
# Parameter in Phantom patch: "comparing" dt_force and dt_drag
DTFORCE_TO_DTDRAG = 1.0

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustybox/accuracy').expanduser()
_paths = sorted(list(root_directory.glob('*')))
paths = {p.name: p for p in _paths}
paths

In [ ]:
hfact = list()
dust_to_gas = list()
C_force = list()
for path in paths.keys():
    eta, eps, cf = [float(s.split('_')[-1]) for s in path.split('-')]
    hfact.append(eta)
    dust_to_gas.append(eps)
    C_force.append(cf)

hfact = sorted(list(set(hfact)))
dust_to_gas = sorted(list(set(dust_to_gas)))
C_force = sorted(list(set(C_force)))
C_drag = [val * DTFORCE_TO_DTDRAG for val in C_force]

In [ ]:
def get(path, q):
    nums = [float(s.split('_')[-1]) for s in path.split('-')]
    if q == 'hfact':
        return nums[0]
    if q == 'dust_to_gas':
        return nums[1]
    if q == 'C_force':
        return nums[2]
    raise ValueError

### Calculate velocity differential time evolution

In [ ]:
data = dict()
exact1 = dict()
exact2 = dict()

for name, path in paths.items():
    try:
        print(f'Running analysis for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        data[name] = dustybox.calculate_differential_velocity(sim)
        exact1[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=True)
        exact2[name] = dustybox.calculate_differential_velocity_exact(sim, backreaction=False)
    except:
        print(f'Failed to load {name}')

### Plot accuracy

In [ ]:
def plot_accuracy(eta):
    _data = {key: val for key, val in data.items() if get(key, 'hfact') == eta}
    _exact1 = {key: val for key, val in exact1.items() if get(key, 'hfact') == eta}
    _exact2 = {key: val for key, val in exact2.items() if get(key, 'hfact') == eta}

    n_dust_to_gas = len(dust_to_gas)
    fig = dustybox.plot_differential_velocity_all(_data, _exact1, _exact2, ncols=2, figsize=(10, 10), transpose=True)
    for ax, eps in zip(fig.axes[:n_dust_to_gas], dust_to_gas):
        ax.set_title(rf'$\epsilon = {eps:.2f}$')
    for ax, fac in zip(fig.axes[::n_dust_to_gas], dtdrag_fac):
        ax.set_ylabel(rf'{fac:.4f}')

In [ ]:
plot_accuracy(eta=1.0)

In [ ]:
plot_accuracy(eta=2.5)

### Calculate velocity differential error

In [ ]:
error = dict()

for name, path in paths.items():
    try:
        print(f'Calculating error for {name}...')
        sim = plonk.load_sim(prefix='dustybox', directory=path)
        error[name] = dustybox.calculate_error(sim, relative=False)
    except:
        print(f'Failed to load {name}')

### Plot error

In [ ]:
def plot_accuracy_error(eta):
    _error = {key: val for key, val in error.items() if get(key, 'hfact') == eta}

    fig = dustybox.plot_error_all(_error, plot_type='linear', ncols=2, figsize=(10, 6), transpose=True)
    for ax, eps in zip(fig.axes[:n_dust_to_gas], dust_to_gas):
        ax.set_title(rf'$\epsilon = {eps:.2f}$')
    for ax, fac in zip(fig.axes[::n_dust_to_gas], dtdrag_fac):
        ax.set_ylabel(rf'{fac:.4f}')

In [ ]:
plot_accuracy_error(eta=1.0)

In [ ]:
plot_accuracy_error(eta=2.5)

In [ ]:
def error_norm_fn(errors, method=2):
    if method == 1:
        return np.sqrt(np.sum([np.mean(err) ** 2 for err in errors]))
    elif method == 2:
        return np.sqrt(np.sum([err ** 2 for err in errors]))
    elif method == 3:
        return np.sum([err for err in errors])
    elif method == 4:
        return np.sum([err.iloc[1] for err in errors])
    raise ValueError

In [ ]:
C_drag = {}
error_norm = {}
for key, val in error.items():
    name = key[:18]
    if C_drag.get(name) is None:
        C_drag[name] = list()
    C_drag[name].append(get(key, 'C_force'))
    if error_norm.get(name) is None:
        error_norm[name] = list()
    error_norm[name].append(error_norm_fn([val['error.1'], val['error.2']], method=2))

In [ ]:
def line(x, m=2, c=0):
    return [m * _x + c for _x in x]

In [ ]:
m, c = 2, -0.1

fig, ax = plt.subplots()

def plot_error_norm(eta, marker, label, ax):
    _C_drag = {key: val for key, val in C_drag.items() if get(key, 'hfact') == eta}
    _error_norm = {key: val for key, val in error_norm.items() if get(key, 'hfact') == eta}

    for (eps, dt), err in zip(_C_drag.items(), _error_norm.values()):
        _eps = get(eps, 'dust_to_gas')
        if label:
            _label = rf'$\eta = {eta}, \epsilon = {_eps}$'
        else:
            _label = None
        ax.plot(np.log10(dt), np.log10(err), marker, label=_label)

    return [np.log10(dt[0]), np.log10(dt[-1])]

plot_error_norm(eta=1.0, marker='d', label=True, ax=ax)
x = plot_error_norm(eta=2.5, marker='s', label=True, ax=ax)
ax.plot(x, line(x, m=m, c=c), '--', color='gray')  #, label=f'Slope {m}')
ax.set(xlabel='log10(dt)', ylabel='log10(error)')
ax.legend()